# NPP Choice

In [2]:
# imports
import xarray as xr
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np 

## HYCOM 0.030 kg m-3

Explanation model calculations (VGPM, CbPM, CAFE) http://orca.science.oregonstate.edu/vgpm.model.php

### WESTBERRY - CbPM


In [3]:
path_west='/data/rd_exchange/sroyer/SEAPOPYM/NPP/1998_2022_NPP_WESTBERRY_CBPM_25KM_8D.nc'
ds_west = xr.open_dataset(path_west,engine='netcdf4')

In [4]:
ds_west

<xarray.Dataset> Size: 10GB
Dimensions:  (time: 1150, lat: 720, lon: 1440)
Coordinates:
  * lon      (lon) float32 6kB -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * lat      (lat) float32 3kB -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * time     (time) datetime64[ns] 9kB 1998-01-01 1998-01-09 ... 2022-12-27
Data variables:
    CbPM     (time, lat, lon) float64 10GB ...

In [5]:
# Select Area of Papa station + time 1998-2020 to be consistent with other forcings
ds_west_papa = ds_west.sel(
    lat=slice(48.5, 50.5),  
    lon=slice(-130.5, -128.5),
    time=slice("1998", "2020"),
)
# Spatial mean
ds_west_papa=ds_west_papa.mean(dim=['lon','lat'],skipna=True)

<div class="alert alert-danger">
    <strong>Warning :</strong> Data every 8 days; For leap years, from march one day difference 
</div>

In [6]:
# Temporal Median
n_periods = 46  # 8-day resolution over 365 days

# create 8 days period index
period_index = np.tile(np.arange(n_periods), int(len(ds_west_papa.time) / n_periods) + 1)[:len(ds_west_papa.time)]
ds_west_papa = ds_west_papa.assign_coords(period_8d=("time", period_index))

# Compute 8_day climatology
clim_8d_west = ds_west_papa.groupby("period_8d").median("time", skipna=True)

# Rename each group 0 45 (group of 8 days) into day in the year (1-365)
day_of_year = [1 + i * 8 for i in range(46)]

# check dims
clim_8d_west = clim_8d_west.assign_coords(day_of_year=("period_8d", day_of_year))

# Rename dims
clim_8d_west = clim_8d_west.swap_dims({"period_8d": "day_of_year"}).drop_vars("period_8d")

In [7]:
# plot 
# Group by year
years = pd.DatetimeIndex(ds_west_papa.time.values).year.unique()

fig = go.Figure()

# Plot one line per year 
for year in years:
    yearly_data = ds_west_papa.sel(time=str(year))["CbPM"]
    fig.add_trace(go.Scatter(
        x=yearly_data.time.dt.dayofyear,
        y=yearly_data.values,
        mode='lines',
        line=dict(width=1),
        name=str(year),
        showlegend=True 
    ))

# Plot climatology in black
fig.add_trace(go.Scatter(
    x=clim_8d_west['day_of_year'],
    y=clim_8d_west['CbPM'].values,
    mode='lines',
    name='Median',
    line=dict(color='black', width=2)
))

fig.update_layout(
    title='8-day CbPM WESTBERRY Primary Productivity Time Series (1998-2020) at PAPA',
    xaxis_title='Day of year',
    yaxis_title='Primary productivity [mg C m-2 day-1]',
    template='plotly_white'
)

fig.show()



To have daily npp like the others forcings and to smooth the curve, lets fit the median with a Fourier serie function.

In [8]:
from scipy.optimize import curve_fit

x_date = clim_8d_west['day_of_year'].values
y_npp = clim_8d_west['CbPM'].values

# Remove NaNs
mask = ~np.isnan(y_npp)
x_clean = x_date[mask]
y_clean = y_npp[mask]

# Fourier series function (order 4)
def fourier_series(x, *params):
    result = params[0]  # a0
    N = (len(params) - 1) // 2
    for n in range(1, N + 1):
        a = params[2 * n - 1]
        b = params[2 * n]
        result += a * np.cos(2 * np.pi * n * x / 365) + b * np.sin(2 * np.pi * n * x / 365)
    return result

order = 4
initial_guess = [np.mean(y_clean)] + [0] * (order * 2) # ao mean of clim, a1, b1, .. an, bn -> 0

# Fit
params, _ = curve_fit(fourier_series, x_clean, y_clean, p0=initial_guess)

# Generate daily values
days = np.arange(365)
west_daily = fourier_series(days, *params)

<div style="padding: 10px; border-radius: 5px; background-color: #cce5ff; color: #004085; border-left: 5px solid #007bff;">
    <strong>Note :</strong> To fit the double peak, a Fourier Serie of order 4 was used.
</div>

In [9]:
# Plot with interpolated daily values

# Group by year
years = pd.DatetimeIndex(ds_west_papa.time.values).year.unique()

fig = go.Figure()

# Plot one line per year 
for year in years:
    yearly_data = ds_west_papa.sel(time=str(year))["CbPM"]
    fig.add_trace(go.Scatter(
        x=yearly_data.time.dt.dayofyear,
        y=yearly_data.values,
        mode='lines',
        line=dict(width=1),
        name=str(year),
        showlegend=True 
    ))

# Plot climatology in black
fig.add_trace(go.Scatter(
    x=clim_8d_west['day_of_year'],
    y=clim_8d_west['CbPM'].values,
    mode='lines',
    name='Median',
    line=dict(color='grey', width=3)
))

# Add the interpolated Fourier daily curve
fig.add_trace(go.Scatter(
    x=np.arange(1, 366),  # day-of-year
    y=west_daily,
    mode='lines',
    name='Fourier Interpolated Median',
    line=dict(color='black', width=3, dash='dash')
))
fig.update_layout(
    title='Daily interpolated CbPM WESTBERRY Primary Productivity Time Series (1998-2020) at PAPA',
    xaxis_title='Day of year',
    yaxis_title='Primary productivity [mg C m-2 day-1]', 
    template='plotly_white'
)

fig.show()


In [10]:
# # Export interpolated daily values of CbPM WESTBERRY npp
# # Create a coordinate for the day of year (1 to 365)
# days = np.arange(1, 366)  # day 1 to 365

# # Create xarray DataArray
# west_daily_da = xr.DataArray(
#     data=west_daily,
#     coords={'days': days},
#     dims=['days'],
#     name='npp'
# )

# west_daily_da.attrs['long_name'] = 'Interpolated daily npp'
# west_daily_da.attrs['units'] = 'mg C m-2 day-1'
# # Export to NetCDF
# west_daily_da.to_netcdf('/data/rd_exchange/sroyer/SEAPOPYM/NPP/daily_WESTBERRY_papa_1998_2020.nc')

### BEHRENFELD -CbPM

In [11]:
path_behr='/data/rd_exchange/sroyer/SEAPOPYM/NPP/1998_2022_NPP_BEHRENFELD_CBPM_25KM_8D.nc'
ds_behr = xr.open_dataset(path_behr,engine='netcdf4')

In [12]:
ds_behr

<xarray.Dataset> Size: 10GB
Dimensions:  (time: 1150, lat: 720, lon: 1440)
Coordinates:
  * lon      (lon) float32 6kB -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * lat      (lat) float32 3kB -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * time     (time) datetime64[ns] 9kB 1998-01-01 1998-01-09 ... 2022-12-27
Data variables:
    CbPM     (time, lat, lon) float64 10GB ...

In [13]:
# Select Area of Papa station + time 1998-2020 to be consistent with other forcings
ds_behr_papa = ds_behr.sel(
    lat=slice(48.5, 50.5),  
    lon=slice(-130.5, -128.5),
    time=slice("1998", "2020"),
)
# Spatial mean
ds_behr_papa=ds_behr_papa.mean(dim=['lon','lat'],skipna=True)

<div class="alert alert-danger">
    <strong>Warning :</strong> Data every 8 days; For leap years, from march one day difference 
</div>

In [14]:
# Temporal Median
n_periods = 46  # 8-day resolution over 365 days

# create 8 days period index
period_index = np.tile(np.arange(n_periods), int(len(ds_behr_papa.time) / n_periods) + 1)[:len(ds_behr_papa.time)]
ds_behr_papa = ds_behr_papa.assign_coords(period_8d=("time", period_index))

# Compute 8_day climatology
clim_8d_behr = ds_behr_papa.groupby("period_8d").median("time", skipna=True)

# Rename each group 0 45 (group of 8 days) into day in the year (1 -365)
day_of_year = [1 + i * 8 for i in range(46)]

# check dims
clim_8d_behr = clim_8d_behr.assign_coords(day_of_year=("period_8d", day_of_year))

# Rename dims
clim_8d_behr = clim_8d_behr.swap_dims({"period_8d": "day_of_year"}).drop_vars("period_8d")

In [15]:
clim_8d_behr

<xarray.Dataset> Size: 736B
Dimensions:      (day_of_year: 46)
Coordinates:
  * day_of_year  (day_of_year) int64 368B 1 9 17 25 33 ... 329 337 345 353 361
Data variables:
    CbPM         (day_of_year) float64 368B nan 148.5 187.0 ... nan nan nan

In [16]:
# plot 
# Group by year
years = pd.DatetimeIndex(ds_behr_papa.time.values).year.unique()

fig = go.Figure()

# Plot one line per year 
for year in years:
    yearly_data = ds_behr_papa.sel(time=str(year))["CbPM"]
    fig.add_trace(go.Scatter(
        x=yearly_data.time.dt.dayofyear,
        y=yearly_data.values,
        mode='lines',
        line=dict(width=1),
        name=str(year),
        showlegend=True 
    ))

# Plot climatology in black
fig.add_trace(go.Scatter(
    x=clim_8d_behr['day_of_year'],
    y=clim_8d_behr['CbPM'].values,
    mode='lines',
    name='Median',
    line=dict(color='black', width=2)
))

fig.update_layout(
    title='8-day CbPM BEHRENFELD Primary Productivity Time Series (1998-2020) at PAPA',
    xaxis_title='Day of year',
    yaxis_title='Primary productivity [mg C m-2 day-1]',
    template='plotly_white'
)

fig.show()



To have daily npp like the others forcings and to smooth the curve, lets fit the median with a Fourier serie function.

In [17]:
from scipy.optimize import curve_fit

x_date = clim_8d_behr['day_of_year'].values
y_npp = clim_8d_behr['CbPM'].values

# Remove NaNs
mask = ~np.isnan(y_npp)
x_clean = x_date[mask]
y_clean = y_npp[mask]

# Fourier series function (order 4)
def fourier_series(x, *params):
    result = params[0]  # a0
    N = (len(params) - 1) // 2
    for n in range(1, N + 1):
        a = params[2 * n - 1]
        b = params[2 * n]
        result += a * np.cos(2 * np.pi * n * x / 365) + b * np.sin(2 * np.pi * n * x / 365)
    return result

order = 4
initial_guess = [np.mean(y_clean)] + [0] * (order * 2) # ao mean of clim, a1, b1, .. an, bn -> 0

# Fit
params, _ = curve_fit(fourier_series, x_clean, y_clean, p0=initial_guess)

# Generate daily values
days = np.arange(365)
behr_daily = fourier_series(days, *params)


<div style="padding: 10px; border-radius: 5px; background-color: #cce5ff; color: #004085; border-left: 5px solid #007bff;">
    <strong>Note :</strong> To fit the double peak, a Fourier Serie of order 4 was used.
</div>

In [18]:
# Plot with interpolated daily values

# Group by year
years = pd.DatetimeIndex(ds_behr_papa.time.values).year.unique()

fig = go.Figure()

# Plot one line per year 
for year in years:
    yearly_data = ds_behr_papa.sel(time=str(year))["CbPM"]
    fig.add_trace(go.Scatter(
        x=yearly_data.time.dt.dayofyear,
        y=yearly_data.values,
        mode='lines',
        line=dict(width=1),
        name=str(year),
        showlegend=True 
    ))

# Plot climatology in black
fig.add_trace(go.Scatter(
    x=clim_8d_behr['day_of_year'],
    y=clim_8d_behr['CbPM'].values,
    mode='lines',
    name='Median',
    line=dict(color='grey', width=3)
))

# Add the interpolated Fourier daily curve
fig.add_trace(go.Scatter(
    x=np.arange(1, 366),  # day-of-year
    y=behr_daily,
    mode='lines',
    name='Fourier Interpolated Median',
    line=dict(color='black', width=3, dash='dash')
))
fig.update_layout(
    title='Daily interpolated CbPM BEHRENFELD Primary Productivity Time Series (1998-2020) at PAPA',
    xaxis_title='Day of year',
    yaxis_title='Primary productivity [mg C m-2 day-1]', 
    template='plotly_white'
)

fig.show()


In [19]:
# # Export interpolated daily values of CbPM BEHRENFELD npp
# # Create a coordinate for the day of year (1 to 365)
# days = np.arange(1, 366)  # day 1 to 365

# # Create xarray DataArray
# behr_daily_da = xr.DataArray(
#     data=behr_daily,
#     coords={'days': days},
#     dims=['days'],
#     name='npp'
# )

# behr_daily_da.attrs['long_name'] = 'Interpolated daily npp'
# behr_daily_da.attrs['units'] = 'mg C m-2 day-1'
# # Export to NetCDF
# behr_daily_da.to_netcdf('/data/rd_exchange/sroyer/SEAPOPYM/NPP/daily_BEHRENFELD_papa_1998_2020.nc')

### CAFE 

In [20]:
path_cafe="/data/rd_exchange/sroyer/SEAPOPYM/NPP/1998_2022_NPP_SILSBE_CAFE_25KM_8D.nc"
ds_cafe = xr.open_dataset(path_cafe,engine='netcdf4')

In [21]:
ds_cafe

<xarray.Dataset> Size: 10GB
Dimensions:  (time: 1150, lat: 720, lon: 1440)
Coordinates:
  * lon      (lon) float32 6kB -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * lat      (lat) float32 3kB -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * time     (time) datetime64[ns] 9kB 1998-01-01 1998-01-09 ... 2022-12-27
Data variables:
    CAFE     (time, lat, lon) float64 10GB ...

In [22]:
# Select Area of Papa station + time 1998-2020 to be consistent with other forcings
ds_cafe_papa = ds_cafe.sel(
    lat=slice(48.5, 50.5),  
    lon=slice(-130.5, -128.5),
    time=slice("1998", "2020"),
)

In [23]:
ds_cafe_papa.CAFE.isnull().sum().item()


10459

In [24]:
# Spatial mean
ds_cafe_papa=ds_cafe_papa.mean(dim=['lon','lat'],skipna=True)

In [25]:
ds_cafe_papa

<xarray.Dataset> Size: 17kB
Dimensions:  (time: 1058)
Coordinates:
  * time     (time) datetime64[ns] 8kB 1998-01-01 1998-01-09 ... 2020-12-26
Data variables:
    CAFE     (time) float64 8kB nan nan 215.2 212.9 199.0 ... nan nan nan nan

<div class="alert alert-danger">
    <strong>Warning :</strong> Data every 8 days; For leap years, from march one day difference 
</div>

In [26]:
# Temporal Median
n_periods = 46  # 8-day resolution over 365 days

# create 8 days period index
period_index = np.tile(np.arange(n_periods), int(len(ds_cafe_papa.time) / n_periods) + 1)[:len(ds_cafe_papa.time)]
ds_cafe_papa = ds_cafe_papa.assign_coords(period_8d=("time", period_index))

# Compute 8_day climatology
clim_8d_cafe = ds_cafe_papa.groupby("period_8d").median("time", skipna=True)

# Rename each group 0 45 (group of 8 days) into day in the year (1 -365)
day_of_year = [1 + i * 8 for i in range(46)]

# check dims
clim_8d_cafe = clim_8d_cafe.assign_coords(day_of_year=("period_8d", day_of_year))

# Rename dims
clim_8d_cafe = clim_8d_cafe.swap_dims({"period_8d": "day_of_year"}).drop_vars("period_8d")

In [27]:
clim_8d_cafe

<xarray.Dataset> Size: 736B
Dimensions:      (day_of_year: 46)
Coordinates:
  * day_of_year  (day_of_year) int64 368B 1 9 17 25 33 ... 329 337 345 353 361
Data variables:
    CAFE         (day_of_year) float64 368B nan 156.6 187.3 ... nan nan nan

In [28]:
# plot 
# Group by year
years = pd.DatetimeIndex(ds_cafe_papa.time.values).year.unique()

fig = go.Figure()

# Plot one line per year 
for year in years:
    yearly_data = ds_cafe_papa.sel(time=str(year))["CAFE"]
    fig.add_trace(go.Scatter(
        x=yearly_data.time.dt.dayofyear,
        y=yearly_data.values,
        mode='lines',
        line=dict(width=1),
        name=str(year),
        showlegend=True 
    ))

# Plot climatology in black
fig.add_trace(go.Scatter(
    x=clim_8d_cafe['day_of_year'],
    y=clim_8d_cafe['CAFE'].values,
    mode='lines',
    name='Median',
    line=dict(color='black', width=2)
))

fig.update_layout(
    title='8-day CAFE Primary Productivity Time Series (1998-2020) at PAPA',
    xaxis_title='Day of year',
    yaxis_title='Primary productivity [mg C m-2 day-1]', 
    template='plotly_white'
)

fig.show()



To have daily npp like the others forcings and to smooth the curve, lets fit the median with a Fourier serie function.

In [29]:
from scipy.optimize import curve_fit

x_date = clim_8d_cafe['day_of_year'].values
y_npp = clim_8d_cafe['CAFE'].values

# Remove NaNs
mask = ~np.isnan(y_npp)
x_clean = x_date[mask]
y_clean = y_npp[mask]

# Fourier series function 
def fourier_series(x, *params):
    result = params[0]  # a0
    N = (len(params) - 1) // 2
    for n in range(1, N + 1):
        a = params[2 * n - 1]
        b = params[2 * n]
        result += a * np.cos(2 * np.pi * n * x / 365) + b * np.sin(2 * np.pi * n * x / 365)
    return result

order = 4
initial_guess = [np.mean(y_clean)] + [0] * (order * 2) # ao mean of clim, a1, b1, .. an, bn -> 0

# Fit
params, _ = curve_fit(fourier_series, x_clean, y_clean, p0=initial_guess)

# Generate daily values
days = np.arange(365)
cafe_daily = fourier_series(days, *params)


In [30]:
cafe_daily

array([148.40534574, 149.53913359, 150.76501109, 152.08107578,
       153.48532592, 154.9756642 , 156.54990182, 158.20576297,
       159.94088974, 161.75284732, 163.6391297 , 165.59716558,
       167.62432479, 169.71792486, 171.87523802, 174.09349842,
       176.36990963, 178.70165236, 181.08589242, 183.51978878,
       186.00050189, 188.52520198, 191.09107756, 193.69534392,
       196.33525161, 199.00809497, 201.71122053, 204.44203535,
       207.19801526, 209.97671284, 212.77576528, 215.59290197,
       218.42595177, 221.27285003, 224.13164521, 227.00050514,
       229.87772286, 232.76172198, 235.65106162, 238.54444078,
       241.44070223, 244.3388358 , 247.2379811 , 250.13742965,
       253.03662638, 255.93517048, 258.83281566, 261.72946963,
       264.62519306, 267.52019778, 270.41484428, 273.30963865,
       276.20522873, 279.10239966, 282.00206878, 284.90527986,
       287.81319675, 290.72709635, 293.64836111, 296.57847084,
       299.5189941 , 302.47157902, 305.43794367, 308.41

In [31]:
# Plot with interpolated daily values

# Group by year
years = pd.DatetimeIndex(ds_cafe_papa.time.values).year.unique()

fig = go.Figure()

# Plot one line per year 
for year in years:
    yearly_data = ds_cafe_papa.sel(time=str(year))["CAFE"]
    fig.add_trace(go.Scatter(
        x=yearly_data.time.dt.dayofyear,
        y=yearly_data.values,
        mode='lines',
        line=dict(width=1),
        name=str(year),
        showlegend=True 
    ))

# Plot climatology in black
fig.add_trace(go.Scatter(
    x=clim_8d_cafe['day_of_year'],
    y=clim_8d_cafe['CAFE'].values,
    mode='lines',
    name='Median',
    line=dict(color='grey', width=3)
))

# Add the interpolated Fourier daily curve
fig.add_trace(go.Scatter(
    x=np.arange(1, 366),  # day-of-year
    y=cafe_daily,
    mode='lines',
    name='Fourier Interpolated Median',
    line=dict(color='black', width=3, dash='dash')
))
fig.update_layout(
    title='Daily interpolated CAFE Primary Productivity Time Series (1998-2020) at PAPA',
    xaxis_title='Day of year',
    yaxis_title='Primary productivity [mg C m-2 day-1]', 
    template='plotly_white'
)

fig.show()


In [ ]:
# Export interpolated daily values of CAFE npp
# Create a coordinate for the day of year (1 to 365)
days = np.arange(1, 366)  # day 1 to 365

# Create xarray DataArray
cafe_daily_da = xr.DataArray(
    data=cafe_daily,
    coords={'days': days},
    dims=['days'],
    name='npp'
)

cafe_daily_da.attrs['long_name'] = 'Interpolated daily npp'
cafe_daily_da.attrs['units'] = 'mg C m-2 day-1'
# Export to NetCDF
#cafe_daily_da.to_netcdf('/data/rd_exchange/sroyer/SEAPOPYM/NPP/daily_CAFE_papa_1998_2020.nc')


### Comparison with paper

In [40]:
dfs = []

for filename, categorie in [
    ("grey_squares.csv", "Grey_Squares"),
    ("white_triangles.csv", "White_Triangles"),
    ("black_circles.csv", "Black_Circles"),
]:
    df = pd.read_csv('/home/sroyer/Desktop/codes/NPP/'+filename)
    df["categorie"] = categorie
    dfs.append(df)

df_paper = pd.concat(dfs, ignore_index=True)
print(df_paper.head())

   day_of_year          nnp     categorie
0          173  1545.015106  Grey_Squares
1          278  1422.356495  Grey_Squares
2          232  1160.120846  Grey_Squares
3          176  1248.942598  Grey_Squares
4          170  1041.691843  Grey_Squares


In [81]:
fig = go.Figure()
# CbPM Westberry
fig.add_trace(go.Scatter(
    x=np.arange(1, 366),  # day-of-year
    y=west_daily,
    mode='lines',
    name='CbPM Westberry climatology',
    line=dict(color='blue', width=3, dash='dash')
))
# CbPM Behrenfeld
fig.add_trace(go.Scatter(
    x=np.arange(1, 366),  # day-of-year
    y=behr_daily,
    mode='lines',
    name='CbPM Behrenfeld climatology',
    line=dict(color='green', width=3, dash='dash')
))
# CAFE
fig.add_trace(go.Scatter(
    x=np.arange(1, 366),  # day-of-year
    y=cafe_daily,
    mode='lines',
    name='CAFE climatology',
    line=dict(color='red', width=3, dash='dash')
))
#Plot points paper

for category in df_paper["categorie"].unique():
    df_cat = df_paper[df_paper["categorie"] == category]
    if category == 'Black_Circles':
        label='Wong et al. (1995)'
        c='black'
    elif category == 'White_Triangles':
        label='Boyd and Harrison (1999)'
        c='lightgrey'
    else:
        label='Welschmeyer et al. (1993)'
        c='grey'
    fig.add_trace(
        go.Scatter(
            x=df_cat["day_of_year"],
            y=df_cat["nnp"],
            mode="markers",
            name=label,
            marker=dict(size=8,color=c),
        )
    )

# Mise en forme
fig.update_layout(
    title='Comparison Primary Productivity Time Series at PAPA',
    xaxis_title='Day of year',
    yaxis_title='Primary productivity [mg C m-2 day-1]', 
    template='plotly_white'
)

# Affichage
fig.show()


In [77]:
# Monthly means

df_paper["month"] = pd.to_datetime(df_paper["day_of_year"], format="%j").dt.month

df_monthly = df_paper.groupby(["categorie", "month"])["nnp"].mean().reset_index()



In [86]:
# Monthly mean npp products
ds_west_papa["month"] = ds_west_papa['time'].dt.month
ds_west_m = ds_west_papa.groupby(["month"]).mean()

ds_behr_papa["month"] = ds_behr_papa['time'].dt.month
ds_behr_m = ds_behr_papa.groupby(["month"]).mean()

ds_cafe_papa["month"] = ds_cafe_papa['time'].dt.month
ds_cafe_m = ds_cafe_papa.groupby(["month"]).mean()

In [83]:
fig = go.Figure()
# CbPM Westberry
fig.add_trace(go.Scatter(
    x=ds_west_m['month'],  
    y=ds_west_m['CbPM'],
    mode='lines',
    name='CbPM Westberry',
    line=dict(color='blue', width=3, dash='dash')
))
# CbPM Behrenfeld
fig.add_trace(go.Scatter(
    x=ds_behr_m['month'],  
    y=ds_behr_m['CbPM'],
    mode='lines',
    name='CbPM Behrenfeld ',
    line=dict(color='green', width=3, dash='dash')
))
# CAFE
fig.add_trace(go.Scatter(
    x=ds_cafe_m['month'],  
    y=ds_cafe_m['CAFE'],
    mode='lines',
    name='CAFE',
    line=dict(color='red', width=3, dash='dash')
))

#Plot points paper

for category in df_monthly["categorie"].unique():
    df_cat = df_monthly[df_monthly["categorie"] == category]
    if category == 'Black_Circles':
        label='Wong et al. (1995)'
        c='black'
    elif category == 'White_Triangles':
        label='Boyd and Harrison (1999)'
        c='lightgrey'
    else:
        label='Welschmeyer et al. (1993)'
        c='grey'
    fig.add_trace(
        go.Scatter(
            x=df_cat["month"],
            y=df_cat["nnp"],
            mode="markers",
            name=label,
            marker=dict(size=8,color=c),
        )
    )

# Mise en forme
fig.update_layout(
    title='Comparison Primary Productivity Monthly Time Series at PAPA (mean)',
    xaxis_title='Month',
    yaxis_title='Primary productivity [mg C m-2 day-1]', 
    template='plotly_white',
    xaxis=dict(tickmode='array', tickvals=list(range(1, 13)), ticktext=[
        'Jan', 'Fev', 'Mar', 'Apr', 'May', 'June', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
    ]),
)

# Affichage
fig.show()


In [ ]:
# Monthly median
df_monthly_md = df_paper.groupby(["categorie", "month"])["nnp"].median().reset_index()

# Monthly median npp products
ds_west_md = ds_west_papa.groupby(["month"]).median()

ds_behr_md = ds_behr_papa.groupby(["month"]).median()

ds_cafe_md = ds_cafe_papa.groupby(["month"]).median()

In [87]:
fig = go.Figure()
# CbPM Westberry
fig.add_trace(go.Scatter(
    x=ds_west_md['month'],  
    y=ds_west_md['CbPM'],
    mode='lines',
    name='CbPM Westberry',
    line=dict(color='blue', width=3, dash='dash')
))
# CbPM Behrenfeld
fig.add_trace(go.Scatter(
    x=ds_behr_md['month'],  
    y=ds_behr_md['CbPM'],
    mode='lines',
    name='CbPM Behrenfeld ',
    line=dict(color='green', width=3, dash='dash')
))
# CAFE
fig.add_trace(go.Scatter(
    x=ds_cafe_md['month'],  
    y=ds_cafe_md['CAFE'],
    mode='lines',
    name='CAFE',
    line=dict(color='red', width=3, dash='dash')
))

#Plot points paper

for category in df_monthly_md["categorie"].unique():
    df_cat = df_monthly_md[df_monthly_md["categorie"] == category]
    if category == 'Black_Circles':
        label='Wong et al. (1995)'
        c='black'
    elif category == 'White_Triangles':
        label='Boyd and Harrison (1999)'
        c='lightgrey'
    else:
        label='Welschmeyer et al. (1993)'
        c='grey'
    fig.add_trace(
        go.Scatter(
            x=df_cat["month"],
            y=df_cat["nnp"],
            mode="markers",
            name=label,
            marker=dict(size=8,color=c),
        )
    )

# Mise en forme
fig.update_layout(
    title='Comparison Primary Productivity Monthly Time Series at PAPA (median)',
    xaxis_title='Month',
    yaxis_title='Primary productivity [mg C m-2 day-1]', 
    template='plotly_white',
    xaxis=dict(tickmode='array', tickvals=list(range(1, 13)), ticktext=[
        'Jan', 'Fev', 'Mar', 'Apr', 'May', 'June', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'
    ]),
)

# Affichage
fig.show()
